In [12]:
from Bio import pairwise2
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
from Bio import SeqIO
import random
import os 
from tqdm import tqdm
from Bio.pairwise2 import format_alignment

from time import time
# synfig == animation
# castle-model-viwer== object 3d 

In [13]:
def Needleman_alignment(seq1, seq2, match=1, mismatch=-1, gap=-2):
    n, m = len(seq1), len(seq2)
    
    
    values = np.zeros((n + 1, m + 1), dtype=int)
    aligns = np.zeros((n + 1, m + 1), dtype=str)
    
    
    values[:, 0] = np.array([gap*i for i in range(n+1)])
    values[0, :] = np.array([gap*i for i in range(m+1)])
    aligns[1:, 0] = "↑"
    aligns[0, 1:] = "←"
    
    
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match_score = values[i - 1, j - 1] + (match if seq1[i - 1] == seq2[j - 1] else mismatch)
            vertucal_score = values[i - 1, j] + gap
            horizontal_score = values[i, j - 1] + gap
            
            values[i, j] = max(match_score, vertucal_score, horizontal_score)
            if values[i, j] == match_score:
                aligns[i, j] = "↖"
            elif values[i, j] == vertucal_score:
                aligns[i, j] = "↑"
            else:
                aligns[i, j] = "←"
    
    
    aligned_seq1, aligned_seq2 = "", ""
    i, j = n, m
    while i > 0 or j > 0:
        print(values[i, j])
        if aligns[i, j] == "↖":
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            i -= 1
            j -= 1
        elif aligns[i, j] == "↑":
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = "-" + aligned_seq2
            i -= 1
        else:  # "←"
            aligned_seq1 = "-" + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1
    print(values)
    print(aligns)

    return aligned_seq1, aligned_seq2, values[n, m]

seq1="AGCT"
seq2="ATGCT"

aligned1, aligned2, score = Needleman_alignment(seq1, seq2,mismatch=-1,gap=-2)
print(aligned1)
print(aligned2)
print(score)

2
1
0
-1
1
[[  0  -2  -4  -6  -8 -10]
 [ -2   1  -1  -3  -5  -7]
 [ -4  -1   0   0  -2  -4]
 [ -6  -3  -2  -1   1  -1]
 [ -8  -5  -2  -3  -1   2]]
[['' '←' '←' '←' '←' '←']
 ['↑' '↖' '←' '←' '←' '←']
 ['↑' '↑' '↖' '↖' '←' '←']
 ['↑' '↑' '↖' '↖' '↖' '←']
 ['↑' '↑' '↖' '↖' '↑' '↖']]
A-GCT
ATGCT
2


In [14]:
def Smith_alignment(seq1, seq2, match=1, mismatch=-1, gap=-2):
    n, m = len(seq1), len(seq2)
    
    
    values = np.zeros((n + 1, m + 1), dtype=int)
    
    
    max_score = 0
    max_pos = (0, 0)
    
    
    for i in range(1, n + 1):
        for j in range(1, m + 1):
            match_score = values[i - 1, j - 1] + (match if seq1[i - 1] == seq2[j - 1] else mismatch)
            vertical_score = values[i - 1, j] + gap
            horizontal_score = values[i, j - 1] + gap
            values[i, j] = max(0, match_score, vertical_score, horizontal_score)  
            
            
            if values[i, j] > max_score:
                max_score = values[i, j]
                max_pos = (i, j)
    
   
    aligned_seq1, aligned_seq2 = "", ""
    i, j = max_pos
    while values[i, j] > 0:
        if values[i, j] == values[i - 1, j - 1] + (match if seq1[i - 1] == seq2[j - 1] else mismatch):
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            i -= 1
            j -= 1
        elif values[i, j] == values[i - 1, j] + gap:
            aligned_seq1 = seq1[i - 1] + aligned_seq1
            aligned_seq2 = "-" + aligned_seq2
            i -= 1
        else:  # values[i, j] == values[i, j - 1] + gap
            aligned_seq1 = "-" + aligned_seq1
            aligned_seq2 = seq2[j - 1] + aligned_seq2
            j -= 1
    
    return aligned_seq1, aligned_seq2, max_score



seq1 = "AAAAAAAAAAAAAAAAAAAAAAGCTAAAAAAaa"
seq2 = "ATGCT"
aligned1, aligned2, score = Smith_alignment(seq1, seq2)
print(aligned1)
print(aligned2)
print(score)

GCT
GCT
3


In [15]:
class Sequence():
    def __init__(self,id,seq,year,variant) -> None:
        self.id = id
        self.seq = seq 
        self.year = year 
        self.variant = variant
    def __len__(self):
        return len(self.seq)
    
data = []
files = os.listdir("Data")
for file in files :
    name = file.split(".")[0]
    variant,year = name.split("_")[0],name.split("_")[1]
    
    for record in SeqIO.parse(f"Data/{file}","fasta"):
       
        data.append(Sequence(record.id,record.seq,variant,year))

In [16]:
seq1,seq2 = data[0],data[3]
aligned1, aligned2, score = Smith_alignment(seq1.seq, seq2.seq)
print(score)

341


In [17]:
seq_data = [d.seq for d in data if len(d) <= 1000]

In [18]:
alignments = pairwise2.align.localxx(seq_data[0],seq_data[1])
print(len(seq_data))

892


In [19]:
class Genetic():
    def __init__(self,data,num_childrens=10,mutation_rate=0.05,mutation_seq=10):
        self.data = data
        self.gen_size = num_childrens
        self.mutation_rate = mutation_rate
        self.mutation_seq = mutation_seq
    
    def cross(self,i,j):
        parents = {"parent1":self.data[i],"parent2":self.data[j]}

        p_size1 = len(parents["parent1"]) 
        p_size2 = len(parents["parent2"])

        p_max = max(p_size1,p_size2)
        align = []
        for c in range(p_max):
            p = np.random.choice([True,False])
            if c < p_size1 and p:
                align.append(parents["parent1"][c])
            elif c < p_size2 and not p :
                align.append(parents["parent2"][c])
            else :
                if c < p_size1 :
                    align.extend(parents["parent1"][c:])
                    break
                else :
                    align.extend(parents["parent2"][c:])
                    break
        return "".join(align)




            
            

        
        
    def mutate(self,protein):
        if np.random.choice([True,False],p=[self.mutation_rate,1-self.mutation_rate]):
            p_size = len(protein)
            start = np.random.randint(0,p_size)
            choices = np.random.choice(['A','G',"T","C","-"],size=self.mutation_seq)
            mutatiion = "".join(choices)

     
            protein = protein[:start]+mutatiion+protein[start+self.mutation_seq:]
        return protein



    
    def find_best(self):
        n = len(self.data)
        pbar = tqdm(total=n*n)
        pbar.set_description("choose Parents")
        best_parents = (0,0)
        best_score = 0
        for i in range(n):
      
            for j in range(n):
                if i!=j:
                    alignments = pairwise2.align.localxx(self.data[i],self.data[j])
            
                    score = alignments[0].score
                    if score > best_score:
                        best_score=score
                        best_parents = (i,j)
                pbar.update(1)
        pbar.close()
        return best_parents,best_score  
      
    def fit(self,iterations):
        t0 = time()
        scores = []
        for iter in range(iterations):
            print(f"Gen : {iter}")
            best_parents,best_score = self.find_best()
            print(best_score)
            scores.append(best_score)
            new_gen = []
            print("make new:")
            for gen in range(self.gen_size):
                new_gen.append(self.mutate(self.cross(*best_parents)))
            self.data = new_gen
        t1 = time()
        best_parents = self.data[best_parents[0]],self.data[best_parents[1]]
        
        return scores,best_parents,best_score,t0-t1


sample = seq_data[0:10]
model = Genetic(sample)



    


In [ ]:
import numpy as np

def hamming_distance_variable_length(s1, s2):
    """
    Calculate the Hamming distance between two sequences of different lengths.
    Only overlapping regions are compared, and non-overlapping regions are penalized.
    """
    # Determine the overlap length
    min_len = min(len(s1), len(s2))
    max_len = max(len(s1), len(s2))

    # Count mismatches in the overlapping region
    mismatch_count = sum(c1 != c2 for c1, c2 in zip(s1[:min_len], s2[:min_len]))

    # Add penalty for non-overlapping regions
    penalty = max_len - min_len

    return mismatch_count + penalty

def hamming_distance_matrix(sequences):
   
    n = len(sequences)
    distance_matrix = np.zeros((n, n), dtype=float)

    # Calculate pairwise distances
    for i in range(n):
        for j in range(i, n):
            distance = hamming_distance_variable_length(sequences[i], sequences[j])
            distance_matrix[i][j] = distance
            distance_matrix[j][i] = distance  # Symmetric matrix

    return distance_matrix

# Example usage
sequences = [
    "ACTG",
    "ACCGG",
    "ATCG",
    "TTT"
]

# Calculate the Hamming distance matrix
hamming_matrix = hamming_distance_matrix(sequences)
print("Hamming Distance Matrix:")
print(hamming_matrix)


Hamming Distance Matrix:
[[0. 2. 2. 3.]
 [2. 0. 2. 5.]
 [2. 2. 0. 3.]
 [3. 5. 3. 0.]]


In [28]:
from Bio.Phylo.TreeConstruction import DistanceTreeConstructor, DistanceMatrix
from Bio import Phylo
import matplotlib.pyplot as plt

# Example condensed distance matrix
names = ["A", "B", "C", "D", "E"]
matrix = [
    [0],
    [17, 0],
    [21, 30, 0],
    [31, 34, 28, 0],
    [23, 21, 39, 43, 0]
]

# Create a DistanceMatrix
dm = DistanceMatrix(names, matrix)

# Initialize a tree constructor
constructor = DistanceTreeConstructor()

# UPGMA Tree
upgma_tree = constructor.upgma(dm)
print("\nUPGMA Tree:")
Phylo.draw_ascii(upgma_tree)

# Neighbor-Joining Tree
nj_tree = constructor.nj(dm)
print("\nNeighbor-Joining Tree:")
Phylo.draw_ascii(nj_tree)



# Draw UPGMA tree
# plt.sca(axes[0])
# Phylo.draw(upgma_tree, do_show=False)
# plt.title("UPGMA Tree")

# # Draw NJ tree
# plt.sca(axes[1])
# Phylo.draw(nj_tree, do_show=False)
# plt.title("Neighbor-Joining Tree")

plt.tight_layout()
plt.show()



UPGMA Tree:
                 ___________________________________________________________ D
  ______________|
 |              |___________________________________________________________ C
_|
 |                           _______________________________________________ E
 |__________________________|
                            |           ____________________________________ B
                            |__________|
                                       |____________________________________ A


Neighbor-Joining Tree:
                                 ___________________________________________ D
              __________________|
  ___________|                  |___________________________ C
 |           |
 |           |___________ A
_|
 |________________ B
 |
 |___________________________________ E



<Figure size 640x480 with 0 Axes>

In [24]:
import numpy as np

def upgma(distance_matrix, labels):
    """
    Perform UPGMA algorithm.
    
    Args:
        distance_matrix (list[list[float]]): A square distance matrix.
        labels (list[str]): List of initial cluster labels.
        
    Returns:
        dict: A hierarchical clustering tree structure.
    """
    n = len(distance_matrix)
    clusters = {i: [labels[i]] for i in range(n)}
    tree = {i: (labels[i], 0) for i in range(n)}  # Store nodes and their heights
    current_index = n

    while len(clusters) > 1:
        # Find the pair with the smallest distance
        min_dist = float('inf')
        pair = (-1, -1)
        for i in range(n):
            for j in range(i + 1, n):
                if distance_matrix[i][j] < min_dist:
                    min_dist = distance_matrix[i][j]
                    pair = (i, j)

        i, j = pair

        # Merge clusters
        new_cluster = clusters[i] + clusters[j]
        clusters[current_index] = new_cluster
        height = min_dist / 2
        tree[current_index] = (new_cluster, height)

        # Update the distance matrix
        new_distances = []
        for k in range(n):
            if k != i and k != j:
                d_new = (distance_matrix[i][k] + distance_matrix[j][k]) / 2
                new_distances.append(d_new)

        distance_matrix = np.delete(distance_matrix, [i, j], axis=0)
        distance_matrix = np.delete(distance_matrix, [i, j], axis=1)
        distance_matrix = np.vstack((distance_matrix, new_distances))
        new_distances.append(0)  # Distance to itself is 0
        distance_matrix = np.column_stack((distance_matrix, new_distances))

        # Remove old clusters
        del clusters[i]
        del clusters[j]

        current_index += 1

    return tree

# Example usage:
distance_matrix = [
    [0, 17, 21, 31, 23],
    [17, 0, 30, 34, 21],
    [21, 30, 0, 28, 39],
    [31, 34, 28, 0, 43],
    [23, 21, 39, 43, 0]
]

labels = ['A', 'B', 'C', 'D', 'E']

result = upgma(distance_matrix, labels)

# Print the resulting tree
for node, (cluster, height) in result.items():
    print(f"Node {node}: Cluster {cluster}, Height {height}")


IndexError: index 4 is out of bounds for axis 0 with size 4

In [ ]:
data = {"method","ram","cpu","time","score","num_seq"}

In [ ]:
model.fit(20)

Gen : 0


choose Parents: 100%|██████████| 100/100 [00:23<00:00,  4.25it/s]


835.0
make new:
Gen : 1


choose Parents: 100%|██████████| 100/100 [00:24<00:00,  4.16it/s]


628.0
make new:
Gen : 2


...

choose Parents: 100%|██████████| 100/100 [00:29<00:00,  3.39it/s]


842.0
make new:
Gen : 19


choose Parents: 100%|██████████| 100/100 [00:39<00:00,  2.52it/s]

842.0
make new:


([835.0,
  628.0,
  719.0,
  781.0,
  817.0,
  834.0,
  842.0,
  842.0,
  842.0,
  842.0,
  842.0,
  842.0,
  842.0,
  842.0,
  842.0,
  842.0,
  842.0,
  842.0,
  842.0,
  842.0],
 (0, 1),
 842.0)

In [ ]:
choices = np.random.choice(['A','G',"T","C","-"],size=5)
mutatiion = "".join(choices)
protein = "ABBBBBDCDD"
protein = protein[:1]+mutatiion+protein[1+5:]
protein

'AAAATCDCDD'

In [ ]:
import random 
import scipy.stats as st 
import numpy as np

event = ["mutation","no mutation"]

ev = np.random.choice(event,p=[0.1,0.9])

print(ev)

mutation
